## import statements

In [51]:
import psycopg2 # postgreSQL's database driver
import numpy as np
import pandas as pd
import sys
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

## variables

In [52]:
DATABASE_INITIAL_ = 'postgres'
DATABASE_ = 'financial'
USER_ = 'postgres'
%run -i C:/Users/champ/Python_proj/postgreSQL_repo/data/vars
PASSWORD_ = password_
HOST_ = '127.0.0.1'
PORT_ = '5432'

## python funtions

In [53]:
def establish_connection(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_):
    
    #establishing the connection
    conn = psycopg2.connect(
        database = DATABASE_,
        user = USER_,
        password = PASSWORD_,
        host = HOST_,
        port = PORT_);
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT);
    return(conn)


def create_database(DATABASE_INITIAL_,
                    DATABASE_,
                    USER_,
                    PASSWORD_,
                    HOST_,
                    PORT_):

    # establishes the connection to the database
    conn = establish_connection(DATABASE_INITIAL_, USER_, PASSWORD_, HOST_, PORT_)

    conn.autocommit = True

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # SQL statement to verify if the dB that is to be created exists alreday
    sql = "SELECT 1 FROM pg_catalog.pg_database WHERE datname = " + "'" + DATABASE_ + "'"
    cursor.execute(sql)
    exists = cursor.fetchone()
    if not exists:
        print('does not exist')
        #Preparing query to create a database
        sql = '''CREATE database '''+DATABASE_;

        #Creating a database
        cursor.execute(sql)
        print(f'\n{DATABASE_} database created successfully........')
        print(f'\nCREATE database commit comment :\n"{sql}"\n')

        # #Closing the connection
        conn.close()
        print(f'\n{DATABASE_} database now closed successfully.')
    else:
        print(f'\nThe dB of the name {DATABASE_} exists already')

        
def detect_table_exists(cursor, TABLE_):
    cursor.execute("select exists(select * from information_schema.tables where table_name=%s)", (TABLE_,))
    if cursor.fetchone()[0]:
        print(f'\nTABLE : "{TABLE_}" EXISTS')
        EXISTS = True
    else:
        print(f'TABLE : "{TABLE_}" DOES NOT EXIST')
        EXISTS = False
    return(EXISTS)


def create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql_script,
                 TABLE_):
    
    # establishes the connection to the database
    conn = establish_connection(DATABASE_, USER_, PASSWORD_, HOST_, PORT_)
    
    conn.autocommit = True

    # Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    # detecting if table exists or not
    TABLE_EXISTS = detect_table_exists(cursor, TABLE_)
    if not TABLE_EXISTS:
        # executing the script
        cursor.execute(sql_script);
        # verification messages
        print(f'\nCREATING TABLE : {TABLE_} ....')
        print(f'\nTable created successfully........')
        print(f'\nCREATE sql commit comment :\n{sql_script}')
    
    # forming the df of the existing table
    arr = ['column_name','data_type', 'is_nullable']
    sql_script = """                              
    SELECT """+arr[0]+""", """+arr[1]+""", """+arr[2]+"""
    FROM information_schema.columns
    WHERE table_name = '"""+TABLE_+"""';""";

    cursor.execute(sql_script);
    print(f'READ sql commit comment :{sql_script}')

    data_ = cursor.fetchall();

    # generating the df
    df = pd.DataFrame(data = data_,  
                            columns = arr)

    print(f'\nPandas schema READ :\n"{df}"\n')

    # closing the connection
    conn.close()
    print(f'{DATABASE_} database now closed successfully.\n\n')


def drop_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 TABLE_):
    '''
    This function drop an existing database table in postgres
    
    '''
    
    # establishes the connection to the database
    conn = establish_connection(DATABASE_, USER_, PASSWORD_, HOST_, PORT_)
    print(f'{DATABASE_} database now opened successfully........')
    
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor();
    
    # detecting if table exists or not
    TABLE_EXISTS = detect_table_exists(cursor, TABLE_)
    if TABLE_EXISTS:
        #Preparing query to create a database
        sql_script = '''DROP TABLE IF EXISTS ''' + TABLE_ + ";"
        cursor.execute(sql_script);
        print(f'{TABLE_} DROP sql commit comment :\n"{sql_script}"')
        print(f"{TABLE_} dropped successfully.")
        
    #Closing the connection
    conn.close()
    print(f'{DATABASE_} database now closed successfully.\n')

    
def update_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql_script,
                 TABLE_):
    
    # establishes the connection to the database
    conn = establish_connection(DATABASE_, USER_, PASSWORD_, HOST_, PORT_)

    conn.autocommit = True

    # creating a cursor object using the cursor() method
    cursor = conn.cursor()
    
    # detecting if table exists or not
    TABLE_EXISTS = detect_table_exists(cursor, TABLE_)
    if TABLE_EXISTS:
        # executing the update script
        cursor.execute(sql_script);

        print(f'\nData inserted successfully........')
        print(f'Table INSERT sql commit comment :\n"{sql_script}"\n')

        # capturing the data from the table
        sql = "SELECT * FROM "+TABLE_
        cursor.execute(sql);
        # obtaining the rows 
        rows = cursor.fetchmany(5);
#         arr = np.array(rows).flatten()
        sql_script = 'Select * FROM '+ TABLE_ + ' LIMIT 0'
        cursor.execute(sql_script);

        # obtaining the column names
        colnames = [desc[0] for desc in cursor.description]

        # constructing a sample of the dataframe
        df = pd.DataFrame(data = rows, columns=colnames)

    #     for i,nr in enumerate(columns):
    #         df.loc[0,columns[nr]] = arr[i]
    #     for i in df.columns:
    #         if i == 'Closed_Date':
    #             df['Closed_Date'] = pd.to_datetime(df['Closed_Date'])

        print(f'\nData fetched successfully........')
        print(f'Table READ LIMIT 5 sql commit comment :\n"{sql}"\n')
        print(f'Pandas display :')
        print(display(df))
        
    # Closing the connection
    conn.close()
    print(f'{DATABASE_} database now closed successfully........')


## creating a database

In [54]:
create_database(DATABASE_INITIAL_,
                DATABASE_,
                USER_,
                PASSWORD_,
                HOST_,
                PORT_)


The dB of the name financial exists already


## creating a table

In [55]:
# create exchange table
TABLE_ = 'exchange'
sql = '''CREATE TABLE IF NOT EXISTS ''' + TABLE_ + ''' (
  id SERIAL PRIMARY KEY,
  name VARCHAR(10) NOT NULL,
  currency CHAR(3) NULL DEFAULT NULL,
  created_date TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
  last_updated  TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP())'''
create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)


# create security table
TABLE_ = 'security'
sql = '''CREATE TABLE IF NOT EXISTS ''' + TABLE_ + ''' (
  id SERIAL PRIMARY KEY,
  exchange_id INTEGER NOT NULL,
  ticker VARCHAR(10) NOT NULL,
  name VARCHAR(100) NULL,
  sector VARCHAR(100) NULL,
  industry VARCHAR(100) NULL,
  created_date TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
  last_updated  TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
    CONSTRAINT fk_exchange_id
        FOREIGN KEY (exchange_id)
        REFERENCES exchange(id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION);'''
create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)


# create data vendor table
TABLE_ = 'data_vendor'
sql = '''CREATE TABLE IF NOT EXISTS ''' + TABLE_ + ''' (
  id SERIAL PRIMARY KEY,
  name VARCHAR(32) NOT NULL,
  website_url VARCHAR(255) NULL DEFAULT NULL,
  created_date TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
  last_updated  TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP());'''
create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)


# create daily_price table
TABLE_ = 'daily_price'
sql = '''CREATE TABLE IF NOT EXISTS ''' + TABLE_ + ''' (
  id SERIAL PRIMARY KEY,
  data_vendor_id INTEGER NOT NULL,
  ticker_id INTEGER NOT NULL,
  price_date DATE NOT NULL,
  created_date TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
  last_updated  TIMESTAMPTZ NULL DEFAULT TRANSACTION_TIMESTAMP(),
  open_price DECIMAL(11,6) NULL DEFAULT NULL,
  high_price DECIMAL(11,6) NULL DEFAULT NULL,
  low_price DECIMAL(11,6) NULL DEFAULT NULL,
  close_price DECIMAL(11,6) NULL DEFAULT NULL,
  adj_close_price DECIMAL(11,6) NULL DEFAULT NULL,
  volume BIGINT NULL DEFAULT NULL,
  CONSTRAINT fk_ticker_id
    FOREIGN KEY (ticker_id)
    REFERENCES security (id)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION,
  CONSTRAINT fk_data_vendor_id
    FOREIGN KEY (data_vendor_id)
    REFERENCES data_vendor (id)
    ON DELETE NO ACTION
    ON UPDATE NO ACTION);'''
create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)


# create realized_profit_loss_tranactions table
TABLE_ = 'realized_profit_loss_tranactions'
sql = '''CREATE TABLE IF NOT EXISTS ''' + TABLE_ + ''' (
Closed_Date DATE NOT NULL,
Symbol CHAR(3) NOT NULL,
Name CHAR(10) NULL,
Quantity INTEGER NULL,
Proceeds FLOAT NULL,
CostBasis FLOAT NULL,
Personal FLOAT NULL,
Total_Gain_Loss_pc FLOAT NULL,
Long_Term_Gain_Loss_amt FLOAT NULL,
Long_Term_Gain_Loss_pc FLOAT NULL,
Short_Term_Gain_Loss_amt FLOAT NULL,
Short_Term_Gain_Loss_pc FLOAT NULL,
Wash_Sale CHAR(3) NULL,
account CHAR(10) NULL
);'''
create_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)



TABLE : "exchange" EXISTS
READ sql commit comment :                              
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns
    WHERE table_name = 'exchange';

Pandas schema READ :
"    column_name                 data_type is_nullable
0            id                   integer          NO
1          name         character varying          NO
2      currency                 character         YES
3  created_date  timestamp with time zone         YES
4  last_updated  timestamp with time zone         YES"

financial database now closed successfully.



TABLE : "security" EXISTS
READ sql commit comment :                              
    SELECT column_name, data_type, is_nullable
    FROM information_schema.columns
    WHERE table_name = 'security';

Pandas schema READ :
"    column_name                 data_type is_nullable
0            id                   integer          NO
1   exchange_id                   integer          NO
2        ticker   

## updating a table

In [56]:
# update exchange table
TABLE_ = 'exchange'
sql = '''INSERT INTO ''' + TABLE_ + ''' (
  name, currency, created_date, last_updated)
  VALUES ('NASDAQ', 'USD', '2000-01-01', '2001-02-01');'''
update_table(DATABASE_,
                 USER_,
                 PASSWORD_,
                 HOST_,
                 PORT_,
                 sql,
                 TABLE_)


TABLE_ = 'realized_profit_loss_tranactions'
sql = '''INSERT INTO ''' + TABLE_ + ''' 
    (Closed_Date,
    Symbol,
    Name,
    Quantity,
    Proceeds,
    CostBasis,
    Personal,
    Total_Gain_Loss_pc,
    Long_Term_Gain_Loss_amt,
    Long_Term_Gain_Loss_pc,
    Short_Term_Gain_Loss_amt,
    Short_Term_Gain_Loss_pc,
    Wash_Sale,
    account
    ) VALUES (
       '2017-06-15',
       'TLT',
       'ISHARES',
       132,
       15847.18,
       16023.15,
       -43.99,
       -0.27,
       0,
       0,
       -43.99,
       -0.27,
       'Yes',
       'Personal'
       );'''
update_table(DATABASE_,
             USER_,
             PASSWORD_,
             HOST_,
             PORT_,
             sql,
             TABLE_)

TABLE_ = 'exchange'
sql = '''INSERT INTO ''' + TABLE_ + ''' (
  name,
  currency,
  created_date,
  last_updated) VALUES (
  'NYSE',
  'USD',
  '2013-01-01',
  '2013-01-01');'''
update_table(DATABASE_,
             USER_,
             PASSWORD_,
             HOST_,
             PORT_,
             sql,
             TABLE_)

TABLE_ = 'security'
sql = '''INSERT INTO ''' + TABLE_ + ''' (
    exchange_id,
    ticker,
    name,
    sector,
    industry,
    created_date,
    last_updated
    ) VALUES (
   1,
   'TLT',
   '10YR',
   'bonds',
   'bonds',
   '2018-04-18',
   '2018-04-18'
   );'''
update_table(DATABASE_,
             USER_,
             PASSWORD_,
             HOST_,
             PORT_,
             sql,
             TABLE_)


TABLE : "exchange" EXISTS

Data inserted successfully........
Table INSERT sql commit comment :
"INSERT INTO exchange (
  name, currency, created_date, last_updated)
  VALUES ('NASDAQ', 'USD', '2000-01-01', '2001-02-01');"


Data fetched successfully........
Table READ LIMIT 5 sql commit comment :
"SELECT * FROM exchange"

Pandas display :


,id,name,currency,created_date,last_updated
0,1,NASDAQ,USD,2000-01-01 00:00:00-08:00,2001-02-01 00:00:00-08:00
1,2,NYSE,USD,2013-01-01 00:00:00-08:00,2013-01-01 00:00:00-08:00
2,3,NASDAQ,USD,2000-01-01 00:00:00-08:00,2001-02-01 00:00:00-08:00


None
financial database now closed successfully........

TABLE : "realized_profit_loss_tranactions" EXISTS

Data inserted successfully........
Table INSERT sql commit comment :
"INSERT INTO realized_profit_loss_tranactions 
    (Closed_Date,
    Symbol,
    Name,
    Quantity,
    Proceeds,
    CostBasis,
    Personal,
    Total_Gain_Loss_pc,
    Long_Term_Gain_Loss_amt,
    Long_Term_Gain_Loss_pc,
    Short_Term_Gain_Loss_amt,
    Short_Term_Gain_Loss_pc,
    Wash_Sale,
    account
    ) VALUES (
       '2017-06-15',
       'TLT',
       'ISHARES',
       132,
       15847.18,
       16023.15,
       -43.99,
       -0.27,
       0,
       0,
       -43.99,
       -0.27,
       'Yes',
       'Personal'
       );"


Data fetched successfully........
Table READ LIMIT 5 sql commit comment :
"SELECT * FROM realized_profit_loss_tranactions"

Pandas display :


,closed_date,symbol,name,quantity,proceeds,costbasis,personal,total_gain_loss_pc,long_term_gain_loss_amt,long_term_gain_loss_pc,short_term_gain_loss_amt,short_term_gain_loss_pc,wash_sale,account
0,2017-06-15,TLT,ISHARES,132,15847.18,16023.15,-43.99,-0.27,0.0,0.0,-43.99,-0.27,Yes,Personal
1,2017-06-15,TLT,ISHARES,132,15847.18,16023.15,-43.99,-0.27,0.0,0.0,-43.99,-0.27,Yes,Personal


None
financial database now closed successfully........

TABLE : "exchange" EXISTS

Data inserted successfully........
Table INSERT sql commit comment :
"INSERT INTO exchange (
  name,
  currency,
  created_date,
  last_updated) VALUES (
  'NYSE',
  'USD',
  '2013-01-01',
  '2013-01-01');"


Data fetched successfully........
Table READ LIMIT 5 sql commit comment :
"SELECT * FROM exchange"

Pandas display :


,id,name,currency,created_date,last_updated
0,1,NASDAQ,USD,2000-01-01 00:00:00-08:00,2001-02-01 00:00:00-08:00
1,2,NYSE,USD,2013-01-01 00:00:00-08:00,2013-01-01 00:00:00-08:00
2,3,NASDAQ,USD,2000-01-01 00:00:00-08:00,2001-02-01 00:00:00-08:00
3,4,NYSE,USD,2013-01-01 00:00:00-08:00,2013-01-01 00:00:00-08:00


None
financial database now closed successfully........

TABLE : "security" EXISTS

Data inserted successfully........
Table INSERT sql commit comment :
"INSERT INTO security (
    exchange_id,
    ticker,
    name,
    sector,
    industry,
    created_date,
    last_updated
    ) VALUES (
   1,
   'TLT',
   '10YR',
   'bonds',
   'bonds',
   '2018-04-18',
   '2018-04-18'
   );"


Data fetched successfully........
Table READ LIMIT 5 sql commit comment :
"SELECT * FROM security"

Pandas display :


,id,exchange_id,ticker,name,sector,industry,created_date,last_updated
0,1,1,TLT,10YR,bonds,bonds,2018-04-18 00:00:00-07:00,2018-04-18 00:00:00-07:00
1,2,1,TLT,10YR,bonds,bonds,2018-04-18 00:00:00-07:00,2018-04-18 00:00:00-07:00


None
financial database now closed successfully........


## deleting/dropping a table

In [57]:
drop_table(DATABASE_,
           USER_,
           PASSWORD_,
           HOST_,
           PORT_,
           'realized_profit_loss_tranactions')

drop_table(DATABASE_,
           USER_,
           PASSWORD_,
           HOST_,
           PORT_,
           'daily_price')

drop_table(DATABASE_,
           USER_,
           PASSWORD_,
           HOST_,
           PORT_,
           'data_vendor')

drop_table(DATABASE_,
           USER_,
           PASSWORD_,
           HOST_,
           PORT_,
           'security')

drop_table(DATABASE_,
           USER_,
           PASSWORD_,
           HOST_,
           PORT_,
           'exchange')

financial database now opened successfully........

TABLE : "realized_profit_loss_tranactions" EXISTS
realized_profit_loss_tranactions DROP sql commit comment :
"DROP TABLE IF EXISTS realized_profit_loss_tranactions;"
realized_profit_loss_tranactions dropped successfully.
financial database now closed successfully.

financial database now opened successfully........

TABLE : "daily_price" EXISTS
daily_price DROP sql commit comment :
"DROP TABLE IF EXISTS daily_price;"
daily_price dropped successfully.
financial database now closed successfully.

financial database now opened successfully........

TABLE : "data_vendor" EXISTS
data_vendor DROP sql commit comment :
"DROP TABLE IF EXISTS data_vendor;"
data_vendor dropped successfully.
financial database now closed successfully.

financial database now opened successfully........

TABLE : "security" EXISTS
security DROP sql commit comment :
"DROP TABLE IF EXISTS security;"
security dropped successfully.
financial database now closed success